# Robertuito
#### Tamara Tlaiye 166912
#### Natalia Rodríguez
#### Renata Contreras 167319

###### Proyecto Final de Machine Learining

In [44]:
#Descargamos librerías necesarias
!pip install accelerate
!pip show accelerate
!pip install pysentimiento
import pandas as pd
import pysentimiento
import pandas as pd
import string
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from pysentimiento import create_analyzer
import torch
import re


Name: accelerate
Version: 0.22.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, packaging, psutil, pyyaml, torch
Required-by: pysentimiento


In [17]:
# Cargarmos los archivos dados en clase
eti = '/content/Libro4.xlsx' #Archivo entrenado
exp = '/content/Libro3.xlsx' #Archivo completo

In [18]:
#Visaulizar archivos
try:
    #que lea archivos de excel
    exp_df = pd.read_excel(exp)
    eti_df = pd.read_excel(eti)

    #head para ver sus primeras filas
    exp_head = exp_df.head()
    eti_head = eti_df.head()

except Exception as e:
    # Para ver si no hay algún error
    exp_head, eti_head = str(e), str(e)

(exp_head, eti_head)

(      PERIODO                   FECHA ESCUELA                  LICENCIATURA  \
 0  OTOÑO 2019 2019-11-18 15:57:07.387    EDAH             ANIMACIÓN DIGITAL   
 1  OTOÑO 2019 2019-11-22 19:26:12.999    EDAH               ARTES PLÁSTICAS   
 2  OTOÑO 2019 2019-11-27 08:25:55.658    EDAH  DISEÑO DE INFORMACIÓN VISUAL   
 3  OTOÑO 2019 2019-11-27 12:35:16.609    EDAH                       IDIOMAS   
 4  OTOÑO 2019 2019-11-27 12:18:05.453    EDAH                       IDIOMAS   
 
                 ORGANIZACION  HORAS  \
 0                  TOKYOPLAY  120.0   
 1  MUSEO REGIONAL DE CHOLULA  200.0   
 2        GURÚ THE BRAND BURÓ  215.0   
 3                       IPAM  100.0   
 4                       IPAM  100.0   
 
    ¿La organización te dio a conocer las reglas, lineamientos y directrices relativas a tus funciones? - 1  \
 0                                                  4                                                         
 1                                                  2 

In [23]:
# Extraemos "Comentarios y/0 sugerencias" de los dos
comments = exp_df['Comentarios y/0 sugerencias']
etiquetados = eti_df['Comentarios y/0 sugerencias']

# Limpiamos texto
def limpiar(text):
    if pd.isna(text): # Si es una cadena de texto
        return ""
    text = text.lower() # Cambiar todo a minúsculas
    text = text.translate(str.maketrans('', '', string.punctuation)) # Eliminamos signos de puntuación
    text = ''.join([i for i in text if not i.isdigit()]) # Eliminamos números
    text = ' '.join(text.split()) # Eliminamos espacios extras
    return text

# Limpiamos
comments_limpios = comments.apply(limpiar)
etiquetados_limpios = etiquetados.apply(limpiar)

# Mostrar los primeros 5 comentarios limpios para verificar
comments_limpios.head(), etiquetados_limpios.head()

(0                                                     
 1    lo que ocurrió es que nunca se autorizo el pre...
 2    fue una excelente experiencia la que viví en p...
 3    la organización no tuvo contacto con nosotras ...
 4    la organización perdió contacto con nosotras a...
 Name: Comentarios y/0 sugerencias, dtype: object,
 0                                                     
 1    lo que ocurrió es que nunca se autorizo el pre...
 2    fue una excelente experiencia la que viví en p...
 3    la organización no tuvo contacto con nosotras ...
 4    la organización perdió contacto con nosotras a...
 Name: Comentarios y/0 sugerencias, dtype: object)

In [25]:
# Archivo de palabras vacías sin "no" y "nada"
lista = '/content/Lista.txt'

# Que lea la lista
with open(lista, 'r', encoding='utf-8') as file:
    palabras = file.readlines()

# Conjunto donde eliminamos duplicados
evitar = set(word.strip() for line in palabras for word in line.split(',') if word.strip())


In [26]:
# Convertir lista a Python
evitar = [word.strip() for line in palabras for word in line.split(',') if word.strip() != '']

#Para búsqueda rápida
evitar = set(evitar)

In [27]:
#Eliminar palabras vacías
def eliminar_palabras(text, stop_words):
    words = text.split() # Texto a palabras
    words_filtrado = [word for word in words if word not in stop_words]  # Filtramos palabras que no están en la lista
    text_filtrado = ' '.join(words_filtrado) # Unir las palabras filtradas de nuevo en un texto
    return text_filtrado

In [28]:
# Quitamos " "
evitar = set(word.replace("'", "").strip() for word in evitar)

In [31]:
# Aplicar función
comentarios_evitar = comments_limpios.apply(lambda x: eliminar_palabras(x, evitar))
etiquetados_evitar = etiquetados_limpios.apply(lambda x: eliminar_palabras(x, evitar))

comentario_ej = comments_limpios.iloc[2]


In [32]:
# Mostrar las primeras 5 palabras vacías limpias y el ejemplo de comentario después de la limpieza
comentario_limpio_ej = eliminar_palabras(comentario_ej, evitar)

list(evitar)[:10], comentario_limpio_ej

(['eran',
  'habremos',
  'tengamos',
  'sí',
  'estaría',
  'estabas',
  'fuiste',
  'hubieses',
  'ser',
  'fuiéramos'],
 'una excelente experiencia viví prácticas profesión organización guru the brand buro realmente creo aprendí mucho y todos equipo involucraron aprendiéramos experiencia')

In [34]:
# Para eliminar comentarios vacíos de comentarios_evitar

# Si es serie
if isinstance(comentarios_evitar, pd.Series):
    # Filtramos
    comment_vacio = comentarios_evitar[comentarios_evitar.str.strip().astype(bool)]
# Si no es una serie
else:
    comentarios_evitar = pd.Series(comentarios_evitar)
    #Filtramos
    comment_vacio = comentarios_evitar[comentarios_evitar.str.strip().astype(bool)]

# Checamos los primeros 5
comment_vacio.head()


1    ocurrió nunca autorizo presupuesto proyecto in...
2    una excelente experiencia viví prácticas profe...
3    organización no tuvo contacto inicio semestre ...
4    organización perdió contacto partir mitad seme...
5    no un contacto continuo parte supervisor dejo ...
Name: Comentarios y/0 sugerencias, dtype: object

In [35]:
# DF con columna
df = pd.DataFrame({'Comments': comentarios_evitar})

# Guardamos en..
archivo = '/content/comments.csv'

# Exportar el DataFrame a CSV
df.to_csv(archivo, index=False)

# Que nos diga donde
archivo

'/content/comments.csv'

In [40]:
# Cargamos el DF
comentarios_df = pd.read_csv("comments.csv")

# Quitar Nan
comentarios_df = comentarios_df.dropna(subset=['Comments'])

In [43]:
# Actualizamos
comentarios_df.reset_index(drop=True, inplace=True)

#Head y shape
comentarios_df.head(), comentarios_df.shape


(                                            Comments
 0  ocurrió nunca autorizo presupuesto proyecto in...
 1  una excelente experiencia viví prácticas profe...
 2  organización no tuvo contacto inicio semestre ...
 3  organización perdió contacto partir mitad seme...
 4  no un contacto continuo parte supervisor dejo ...,
 (1631, 1))

In [41]:
# Modelo que vamos a usar
modelo = "pysentimiento/robertuito-base-uncased"

# Cargamos model
modelo1 = AutoModelForSequenceClassification.from_pretrained(modelo, num_labels=3)

#Cargamos tokenizador
tokenizador = AutoTokenizer.from_pretrained(modelo)

In [42]:
# Set the model max length if needed (based on the guide from the notebook)
tokenizador.model_max_length = 128

# Just to ensure that the model and tokenizer are loaded correctly, print the model and tokenizer
modelo1, tokenizador

(RobertaForSequenceClassification(
   (roberta): RobertaModel(
     (embeddings): RobertaEmbeddings(
       (word_embeddings): Embedding(30000, 768, padding_idx=1)
       (position_embeddings): Embedding(130, 768, padding_idx=1)
       (token_type_embeddings): Embedding(1, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): RobertaEncoder(
       (layer): ModuleList(
         (0-11): 12 x RobertaLayer(
           (attention): RobertaAttention(
             (self): RobertaSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): RobertaSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bia

In [45]:
# Sentiment Analysis
sa = create_analyzer(task="sentiment", lang="es")

# Para predecir el sentiment
def predict_sa(comentario):
    return sa.predict(comentario)

# Aplica la predicción a cada comentario
comentarios_df['Sentiment'] = comentarios_df['Comments'].apply(predict_sa)

# Guarda los resultados en un nuevo archivo CSV
comentarios_df.to_csv("comments_sentimientos.csv", index=False)


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [51]:
# Extraemos sentimiento y la mayor probabilidad de string
def extraemos(output):
    #si isna
    if pd.isna(output):
        return None, None
    sentiment = re.search(r"output=([A-Z]+)", output)
    proba = re.search(r"probas={([^}]+)}", output)
    #
    if sentiment and proba:
        sentimiento = sentiment.group(1)
        proba_str = proba.group(1)
        probas = dict((k.strip(), float(v.strip())) for k, v in
                      (pair.split(':') for pair in proba_str.split(',')))
        maximo = max(probas.values())
        return sentimiento, max
    else:
        return None, None


In [52]:
#Cargamos el archivo
sentiment_df = pd.read_csv("comments_sentimientos.csv")

# Aplicamos la función
sentiment_info = sentiment_df['Sentiment'].apply(lambda x: extraemos(x))

# Creamos columnas sentimiento y prob maxima
sentiment_df['Sentiment_Limpio'] = sentiment_info.apply(lambda x: x[0] if x else None)
sentiment_df['Proba'] = sentiment_info.apply(lambda x: x[1] if x else None)

# Cargamos Libro4 (Experiencias original)
exp_df = pd.read_excel("Libro3.xlsx")

# Checar si comentarios en el mismo orden
sentiment_df['ORGANIZACION'] = exp_df['ORGANIZACION']

# Agrupamos  comentarios por empresa y frecuencia de sentimientos
resultados = sentiment_df.groupby('ORGANIZACION')['Sentiment_Limpio'].value_counts().unstack().fillna(0)

# Identificamos las 5 organizaciones con más comentarios positivos, negativos y neutrales
top_pos = resultados['POS'].sort_values(ascending=False).head(5)
top_neg = resultados['NEG'].sort_values(ascending=False).head(5)
top_neu = resultados['NEU'].sort_values(ascending=False).head(5)


In [56]:
# Imprimimos los resultados
print("Top 5 organizaciones con comentarios positivos:\n", top_pos)
print("\n")
print("Top 5 organizaciones con comentarios negativos:\n", top_neg)
print("\n")
print("Top 5 organizaciones con comentarios neutrales:\n", top_neu)
print("\n")

Top 5 organizaciones con comentarios positivos:
 ORGANIZACION
ACCURATE MEDIA      15.0
OZONO POLARIS       10.0
PROYECTO INTERNO    10.0
PIXOLAB              9.0
BASE 10              9.0
Name: POS, dtype: float64


Top 5 organizaciones con comentarios negativos:
 ORGANIZACION
UNIVERSIDAD AUTÓNOMA DE BAJA CALIFORNIA                                               5.0
HOSPITAL ANGELES PUEBLA                                                               4.0
COLEGIO LUMINOS                                                                       4.0
BIRTH GROUP                                                                           4.0
DESARROLLO DE PROYECTOS DEL DEPARTAMENTO DE COMPUTACIÓN, ELECTRÓNICA Y MECATRÓNICA    3.0
Name: NEG, dtype: float64


Top 5 organizaciones con comentarios neutrales:
 ORGANIZACION
ACCURATE MEDIA                                              9.0
1ER. BOOT CAMP IBEROAMERICANO DE INNOVACIÓN ABIERTA 2021    7.0
FAURECIA                                               